In [1]:
import numpy as np

In [2]:
TOTAL_DURATION = 450.
LEAD_IN_DURATION = 6.
EQUATION_DURATION = 2.
ISI1 = 0.5
COMPARISON_DURATION = 5.
ISI2 = 0.5
FEEDBACK_DURATION = 2.
ITI = 0.5
END_SCREEN_DURATION = 6.

duration_left = TOTAL_DURATION - (LEAD_IN_DURATION + END_SCREEN_DURATION)

In [3]:
# parameters to test
ranges = {
    'equation': (2, 4),
    'isi1': (0.5, 2.5),
    'comparison': (3, 7),
    'isi2': (0.5, 2.5),
    'feedback': (1, 3),
    'iti': (0.5, 5),
}

n_trials = 29

In [4]:
c = 0
total_trial_duration = np.inf
while total_trial_duration > duration_left:
    durations = {}
    for stage, range_ in ranges.items():
        # using uniform distributions...
        durations[stage] = np.random.randint(range_[0]*10, range_[1]*10, size=n_trials) / 10
    dur_arr = np.stack(list(durations.values()))
    total_trial_duration = dur_arr.sum()
    c += 1
    if c > 1000:
        print('No solution found.')
        break
print('{} / {}'.format(dur_arr.sum(), duration_left))

427.6 / 438.0


In [5]:
durations

{'comparison': array([5.1, 6.4, 6.8, 5.8, 4. , 4.7, 5.3, 3.6, 4.5, 4.8, 3.1, 4.7, 6.8,
        4.3, 4. , 4.1, 3.1, 6.6, 5.1, 5. , 6.5, 4.3, 5.4, 3.5, 3. , 6.8,
        6.5, 3.9, 4. ]),
 'equation': array([3.7, 3.1, 3.2, 3.9, 3.2, 2.6, 2.7, 3.3, 3. , 2.3, 3.3, 3.1, 3.2,
        3.4, 2.3, 3.6, 3.4, 2. , 2.6, 3.7, 3.6, 2.7, 3.6, 2.5, 3.2, 2.1,
        2.8, 3. , 2.5]),
 'feedback': array([1.5, 1.1, 2.7, 1.8, 2.2, 1.8, 1.3, 1.5, 1.4, 1.6, 2.2, 2. , 2.3,
        1. , 1.1, 1. , 2.4, 2. , 2. , 2.7, 1.1, 1.4, 1.6, 1.1, 1.7, 1.4,
        1.7, 1.7, 1.3]),
 'isi1': array([1.3, 0.7, 0.9, 2. , 0.9, 1.5, 0.5, 2.3, 2.2, 0.6, 0.7, 0.7, 0.9,
        2.1, 1.8, 0.7, 2. , 2.3, 1.8, 1.7, 1.8, 1.1, 1.1, 1.2, 1.6, 2.3,
        0.9, 1.6, 0.6]),
 'isi2': array([1.2, 1.7, 1.2, 2.2, 1.4, 1.2, 1.1, 0.9, 2.2, 2.4, 1. , 0.5, 0.5,
        2. , 1.3, 2.3, 0.6, 2.3, 1.6, 1.4, 0.6, 2. , 0.7, 1. , 2.1, 0.5,
        1.5, 0.6, 0.7]),
 'iti': array([4.2, 4.5, 0.5, 1.7, 1.6, 4.7, 1.6, 2. , 1.4, 2.4, 0.6, 0.7, 3.5,
        4.1